In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/mbti-type/mbti_1.csv')
df.head()

In [ ]:
df=df.dropna()
df.reset_index(inplace=True)

x=df['posts']

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

y = df.iloc[:,1].values

le=LabelEncoder()
y=le.fit_transform(y)

y=to_categorical(y)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [ ]:
voc_size = 50000
messages=x.copy()
messages



In [ ]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range (0,len(messages)):
    review = re.sub('[^a-zA-Z]','',messages[i])
    review=review.lower()
    review=review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english') ]
    review=''.join(review)
    corpus.append(review)

In [ ]:
oe=[one_hot(words,voc_size) for words in corpus ]

In [ ]:
sent_length=250
embedded_docs = pad_sequences(oe,padding='pre',maxlen = sent_length)

In [ ]:
dimension=100
model = Sequential()
model.add(Embedding(voc_size,dimension,input_length = sent_length))
model.add(Dropout(0.25))
model.add(LSTM(100))
model.add(Dropout(0.25))
model.add(Dense(16,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer ='adam',metrics=['accuracy'])

In [ ]:

X=np.array(embedded_docs)


In [ ]:
df.type.value_counts()

In [ ]:
import imblearn
# import library
from imblearn.under_sampling import RandomUnderSampler

from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42) # fit predictor and target variable
x_rus, y_rus = ros.fit_resample(X, y)


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_rus, y_rus, test_size=0.2, random_state=42)

In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=8,batch_size=64)

In [ ]:
model.save('Prediction.h5')

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from tensorflow.keras.preprocessing.text import one_hot
from nltk.stem.porter import PorterStemmer

def preprocess(tweet):
    
    ps=PorterStemmer()
    
    L=[]

    review = re.sub('[^a-zA-Z]','',tweet[0])
    review=review.lower()
    review=review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english') ]
    review=''.join(review)
    L.append(review)
    
    oe=[one_hot(words,voc_size) for words in L]
    
    sent_length=250
    doc = pad_sequences(oe,padding='pre',maxlen = sent_length)
    
    return doc


In [ ]:
tweet=[' i am jew']
pred=preprocess(tweet)
y_pred=model.predict(pred)
y_pred=np.argmax(y_pred)

print(y_pred)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()